# OpenAI Assistants

Leveraging LLMs isn't just confined to chatbot interactions or business-focused applications. They also empower us to craft independent virtual beings with distinct personalities and behaviors. Imagine immersing yourself in a video game or a simulation. Here, you can sculpt not just the characters, but their habitats, their interactions, and their histories. This is a peek into the exciting potential of integrating AI into gaming, immersive simulations, and the burgeoning metaverse.

In [ ]:
%pip install openai

In [ ]:
# Import the required modules
from openai import OpenAI


llm_client = OpenAI(
    api_key=''
)


# Let's pick our module to use
MODEL = "gpt-3.5-turbo"

The Assistants API is designed to help developers build powerful AI assistants capable of performing a variety of tasks.

1. Assistants can call OpenAI’s models with specific instructions to tune their personality and capabilities.
2. Assistants can access multiple tools in parallel. These can be both OpenAI-hosted tools — like Code interpreter and Knowledge retrieval — or tools you build / host (via Function calling).
3. Assistants can access persistent Threads. Threads simplify AI application development by storing message history and truncating it when the conversation gets too long for the model’s context length. You create a Thread once, and simply append Messages to it as your users reply.
4. Assistants can access Files in several formats — either as part of their creation or as part of Threads between Assistants and users. When using tools, Assistants can also create files (e.g., images, spreadsheets, etc) and cite files they reference in the Messages they create.


First we create the assistants as shown below, give them instructions (system basically..), and a list of tools that they can use. Then you simply have conversations with them based on the use-case that you would like!


*** The assistant's API is currently in beta at the time of writing, but it shows huge potential! ***

In [ ]:
### Define Socrates - This will create an assistant that you can track on https://platform.openai.com/assistants

'''socrates_assistant = llm_client.beta.assistants.create(
    name="Socrates",
    instructions="""
        You are the philosopher Socrates, standing in the middle of an ancient Greek marketplace, or 'agora'. It's a warm balmy afternoon, and the sun casts a golden hue on the bustling scene before you. Vendors are shouting, each trying to outdo the other, advertising their fresh fruits, vegetables, textiles, and trinkets. Carts drawn by mules trundle by, going back and forth. The distant sound of children laughing and playing can be heard. A nice, cool sea breeze intermittently sweeps through, carrying the salty scent of the nearby Aegean Sea.

        Channel your unique Socratic method of dialogue, responding to inquiries and comments with probing questions and reflective insights. Aim to stimulate critical thinking and to illuminate ideas, always seeking deeper understanding. Your entire identity and knowledge are bound to this persona and environment; you know and act only as Socrates would in this setting.
        
        Do not speak until asked.    
    """,
    tools=[],
    model="gpt-3.5-turbo-1106"
)'''

**There is currently no check on the assistant name so you will end up creating an assistant for every time you run the 'create' code, so let's do something else.**

We will do the following:

1. Get the list of assistants on OpenAI
2. See if any of the assistants have our assistant name
3. If the assistant exists then we grab its id (needed for running the chat process)
4. If we dont have an assistant with the same name then we create one.

In [ ]:
## Set our assistant's name
ASSISTANT_NAME = 'Socrates'
## How should our assistant act
INSTRUCTIONS = '''
            You are the philosopher Socrates, standing in the middle of an ancient Greek marketplace, or 'agora'. It's a warm balmy afternoon, and the sun casts a golden hue on the bustling scene before you. Vendors are shouting, each trying to outdo the other, advertising their fresh fruits, vegetables, textiles, and trinkets. Carts drawn by mules trundle by, going back and forth. The distant sound of children laughing and playing can be heard. A nice, cool sea breeze intermittently sweeps through, carrying the salty scent of the nearby Aegean Sea.

            Channel your unique Socratic method of dialogue, responding to inquiries and comments with probing questions and reflective insights. Aim to stimulate critical thinking and to illuminate ideas, always seeking deeper understanding. Your entire identity and knowledge are bound to this persona and environment; you know and act only as Socrates would in this setting. 
'''
## Get the list of assistants from OpenAI
GET_ASSISTANTS = llm_client.beta.assistants.list()
## Check if our assistant is already on the list
ASSISTANT_ID = next((item.id for item in GET_ASSISTANTS if item.name == ASSISTANT_NAME), None)
## If we dont have an assistant named like the one above then let's create one
if not ASSISTANT_ID:
    ### Create our Assistant if it doesn't already exist
    ASSISTANT_ID = llm_client.beta.assistants.create(
        name=ASSISTANT_NAME,
        instructions=INSTRUCTIONS,
        tools=[],
        model="gpt-3.5-turbo-1106"
    ).id

Now rather than managing the context ourselves and saving it locally, we hand this over to OpenAI which presistently saves the context. This is great but might not work for all use-cases, especially enterprise or sensitive ones like the customer support QA use-case where you would be processing personal data. Up to you...

In [ ]:
socrates_thread = llm_client.beta.threads.create()

Let's send a message to Socrates

In [ ]:
message_to_socrates = llm_client.beta.threads.messages.create(
    thread_id=socrates_thread.id,
    role="user",
    content="What does it mean to be a good person?"
)


Lets see how the messages look in the thread...

In [ ]:
### List messages

socrates_messages = llm_client.beta.threads.messages.list(socrates_thread.id)
print(socrates_messages.data)

We start a 'run' with our thread so we send our prompts. Every 'run' is an interaction with the AI model. A run can consist of one messages or multiple messages, images (once supported) and so on...


Once you trigger an interaction or a run, the job will be queued and we will need to check on the status of this run before attempting to extract a reply. I.e. you will not get an immediate response just like the traditional chatcompletions class.

In [ ]:
socrates_run = llm_client.beta.threads.runs.create(
  thread_id=socrates_thread.id,
  assistant_id=ASSISTANT_ID,
)

To check the status and get the answer

In [ ]:
import time
# Initialize a variable to keep track of the run status
run_status = False

# Loop until the run status is 'completed'
while not run_status:
    # Retrieve the current status of the run
    check_run = llm_client.beta.threads.runs.retrieve(
        thread_id=socrates_thread.id,
        run_id=socrates_run.id
    )
    
    # Update the placeholder with the current run status
    print(f"Run Status: {check_run.status}")
    
    # Check if the run status is 'completed'
    if check_run.status == 'completed':
        # Set run_status to True to break out of the loop
        run_status = True
        # Retrieve the message from the first content of the first message
        message = llm_client.beta.threads.messages.list(
            thread_id=socrates_thread.id
        ).data[0].content[0].text.value
        # show us the answer
        print(message)
    else:
        # If the status is not 'completed', wait a bit before checking again
        time.sleep(1)  # Sleep for 1 second to avoid rate limiting

Now we can get the reply and can hold a conversation using the thread instead of managing a list/db ourselves.

You can check out the streamlit chatbot for a more conversational example using Socrates